<a href="https://colab.research.google.com/github/Mahnazshamissa/Python/blob/main/asigmo_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup notebook environment

Make sure the Colab environment has a GPU enabled *Edit->Notebook Settings->Hardware accelerator->Choose GPU*

In [ ]:
! nvidia-smi

Thu Oct 29 09:28:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Download the dataset first

In [ ]:
! wget https://pub.ist.ac.at/~mlechner/datasets/f8d.tar.gz
! tar -xf f8d.tar.gz

--2020-10-29 09:28:51--  https://pub.ist.ac.at/~mlechner/datasets/f8d.tar.gz
Resolving pub.ist.ac.at (pub.ist.ac.at)... 81.223.84.195
Connecting to pub.ist.ac.at (pub.ist.ac.at)|81.223.84.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76149807 (73M) [application/x-gzip]
Saving to: ‘f8d.tar.gz’

f8d.tar.gz          100%[===================>]  72.62M  9.03MB/s    in 9.0s    

2020-10-29 09:29:00 (8.07 MB/s) - ‘f8d.tar.gz’ saved [76149807/76149807]



Import python libraries

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Step 1: Define datasets

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "f8d/test", shuffle=False, batch_size=128, image_size=(256, 256),
)
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "f8d/training",
  shuffle=True,
  batch_size=64,
  image_size=(256, 256),
  validation_split=0.2,
  subset="training",
  seed=1020202,
)
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "f8d/training",
  shuffle=True,
  batch_size=128,
  image_size=(256, 256),
  validation_split=0.2,
  subset="validation",
  seed=1020202,
)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

Found 640 files belonging to 8 classes.
Found 1840 files belonging to 8 classes.
Using 1472 files for training.
Found 1840 files belonging to 8 classes.
Using 368 files for validation.


## Step 2 and 3: Define model and objective

In [ ]:
pretrained_model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
# include_top=False means cutting the output layer

model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(256,256,3)),
  tf.keras.layers.experimental.preprocessing.RandomCrop(224, 224),
  tf.keras.layers.Lambda(lambda x: tf.keras.applications.resnet.preprocess_input(x)),
  pretrained_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(8, activation="softmax"),
])
pretrained_model.trainable=True
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_crop_2 (RandomCrop)   (None, 224, 224, 3)       0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 16392     
Total params: 23,604,104
Trainable params: 23,550,984
Non-trainable params: 53,120
_________________________________________________________________


## Step 4: Train the model

In [ ]:
training_log = model.fit(
  train_dataset, epochs=20, validation_data=valid_dataset
)

Epoch 1/20
23/23 [==============================] - 15s 673ms/step - loss: 1.3393 - sparse_categorical_accuracy: 0.6318 - val_loss: 5020.7368 - val_sparse_categorical_accuracy: 0.1549
Epoch 2/20
23/23 [==============================] - 15s 658ms/step - loss: 0.8454 - sparse_categorical_accuracy: 0.7615 - val_loss: 307.9615 - val_sparse_categorical_accuracy: 0.1168
Epoch 3/20
23/23 [==============================] - 15s 664ms/step - loss: 0.4314 - sparse_categorical_accuracy: 0.8662 - val_loss: 15.4648 - val_sparse_categorical_accuracy: 0.2582
Epoch 4/20
23/23 [==============================] - 15s 670ms/step - loss: 0.2449 - sparse_categorical_accuracy: 0.9198 - val_loss: 6.0629 - val_sparse_categorical_accuracy: 0.2853
Epoch 5/20
23/23 [==============================] - 16s 677ms/step - loss: 0.1327 - sparse_categorical_accuracy: 0.9613 - val_loss: 2.0494 - val_sparse_categorical_accuracy: 0.5326
Epoch 6/20
23/23 [==============================] - 16s 682ms/step - loss: 0.0917 - spars

In [ ]:
model.save_weights("my_model.h5")

In [ ]:
print(f"Best validation accuracy is {100*np.max(training_log.history['val_sparse_categorical_accuracy']):0.2f}%")

Best validation accuracy is 95.38%


## Step 5: Evaluate the model

In [ ]:
model.evaluate(test_dataset)

5/5 [==============================] - 2s 318ms/step - loss: 0.2021 - sparse_categorical_accuracy: 0.9391


[0.2020607888698578, 0.9390624761581421]

In [ ]:
pretrained_model.summary()

In [ ]:
tf.keras.utils.plot_model(pretrained_model)

In [ ]:
pretrained_model.trainable = True
for l in pretrained_model.layers:
  l.trainable=False
for i in range(10):
  pretrained_model.layers[-i].trainable=True
  
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_crop_1 (RandomCrop)   (None, 224, 224, 3)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 16392     
Total params: 23,604,104
Trainable params: 3,432,968
Non-trainable params: 20,171,136
_________________________________________________________________


In [ ]:
training_log2 = model.fit(
  train_dataset, epochs=7, validation_data=valid_dataset
)

Epoch 1/7
23/23 [==============================] - 6s 278ms/step - loss: 0.0010 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.1453 - val_sparse_categorical_accuracy: 0.9511
Epoch 2/7
23/23 [==============================] - 7s 283ms/step - loss: 6.7090e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.1518 - val_sparse_categorical_accuracy: 0.9538
Epoch 3/7
23/23 [==============================] - 7s 290ms/step - loss: 4.4333e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.1435 - val_sparse_categorical_accuracy: 0.9484
Epoch 4/7
23/23 [==============================] - 7s 291ms/step - loss: 3.0655e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.1515 - val_sparse_categorical_accuracy: 0.9429
Epoch 5/7
23/23 [==============================] - 7s 295ms/step - loss: 3.5091e-04 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.1253 - val_sparse_categorical_accuracy: 0.9620
Epoch 6/7
14/23 [=================>............] - ETA: 1s - loss: 2.8272e-04 - sparse_ca

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(test_dataset)

5/5 [==============================] - 2s 313ms/step - loss: 0.2032 - sparse_categorical_accuracy: 0.9344


[0.2031702995300293, 0.934374988079071]